# Здесь мы парсим xml

In [35]:
# os
import os
# регулярки.
import re
# удобная штука для того, чтобы понимать, сколько осталось в цикле.
from tqdm.notebook import tqdm
# pretty print
from pprint import pprint
# токенизация
from nltk import sent_tokenize
# нужен html.parser, только для xml
from bs4 import BeautifulSoup

In [41]:
files = os.listdir('xml_data/')

In [24]:
# берет xml-файл, выдает только текст книги, разделенный тэгом {par} по абзацам
def par_tag_clean_text(filename):
    
    print('Stripping the lines...')
    with open(filename, encoding='utf-8') as f:
        text = f.read()
    n_text = ''
    for line in tqdm(text.split('\n')):
        n_line = line.strip()
        n_text += n_line+'\n'
    text = n_text
    soup = BeautifulSoup(text, 'html.parser')
    all_pars = soup.find_all('p', {'rend': 'left'})
    prev_pars = []
    real_pars = []
    
    print('Deleting unnecessary paragraphs...')
    for p1 in tqdm(all_pars):
        p1_in_prev = False
        for p2 in prev_pars[::-1]:
            if str(p1) in str(p2):
                p1_in_prev = True
                break
        if p1_in_prev == False:
            real_pars.append(str(p1)[15:-4])
        prev_pars.append(str(p1))
    
    par_text = '{par}'.join(real_pars)
    par_soup = BeautifulSoup(text, 'html.parser')
    
    notes = par_soup.find_all('note')
    pbs = par_soup.find_all('pb')
    
    print('Deleting notes and pb tags...')
    for note in tqdm(notes):
        par_text = par_text.replace(str(note), '')
    for pb in tqdm(pbs):
        par_text = par_text.replace(str(pb), '')
    
    print('Deleting highlight tags...')
    his = ('</hi>', '<hi rend="italic">', '<hi rend="sup">', '<hi rend="bold">')
    for hi in tqdm(his):
        par_text = par_text.replace(hi, '')
    par_text = re.sub('<pb n=".*?" \/>', '', par_text)
    
    print('Done!')
    
    return par_text

In [30]:
# берет этот же текст, достает из него все строчки диалогов
def extract_dialogue(text):
    
    print('Extracting dialogue...')
    pars = text.split('{par}')
    dialogue = []
    for p in tqdm(pars):
        if p.strip().startswith('—'):
            dialogue.append(p.strip())
    
    return dialogue

In [11]:
# пишет диалоги в файлы
def write_dialogue(path, filename, dialogue):
    print('Writing dialogue...')
    with open(path+filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(dialogue))
    print('Success!')

In [33]:
for file in files:
    print(file+':')
    text = par_tag_clean_text(f'xml_data/{file}')
    dialogue = extract_dialogue(text)
    write_dialogue('corpus/', os.path.splitext(file)[0]+'.txt', dialogue)

Детство 1:
Stripping the lines...



Deleting unnecessary paragraphs...



Deleting notes and pb tags...



Deleting highlight tags...



Done!
Extracting dialogue...



Writing dialogue...
Success!
